In [1]:
import pybaseball
from pybaseball import statcast_pitcher
from pybaseball import playerid_lookup
from pybaseball import batting_stats
from bs4 import BeautifulSoup
import requests
import pandas as pd
import matplotlib.pyplot as plt
import pymongo

In [3]:
stat_data2018 = pd.read_csv("resources/sprint_speed_2018.csv")
stat_data2018 = stat_data2018.rename(index=str, columns= {"player_id": "key_mlbam"})
stat_data2019 = pd.read_csv("resources/sprint_speed2019.csv")
stat_data2019 = stat_data2019.rename(index=str, columns= {"player_id": "key_mlbam"})

In [4]:
# retrieve data on only players who have 50+ plate appearances this year
bat_data2018 = batting_stats(2018, qual=50)

In [5]:
fangraphsids_db = pybaseball.playerid_reverse_lookup(stat_data2018.key_mlbam, key_type='mlbam')

Gathering player lookup table. This may take a moment.


In [14]:
labelled = stat_data2018.merge(fangraphsids_db, how='left', on='key_mlbam' )
labelled = labelled.drop(columns=['team', 'age'])

In [16]:
labelled['Name'] = labelled['first_name'] + " " + labelled['last_name']

In [17]:
total2018 = labelled.merge(bat_data2018, how='left', on='Name')

In [ ]:
CFers = total2018[total2018.position.str.endswith('F')]

In [29]:
total2018.head()

,last_name,first_name,key_mlbam,team_id,position,competitive_runs,hp_to_1b,sprint_speed,name_last,name_first,...,wSL/C (pi),wXX/C (pi),O-Swing% (pi),Z-Swing% (pi),Swing% (pi),O-Contact% (pi),Z-Contact% (pi),Contact% (pi),Zone% (pi),Pace (pi)
0,Buxton,Byron,621439,142,CF,35,3.98,30.5,buxton,byron,...,-1.59,NaN,0.367,0.694,0.532,0.554,0.832,0.737,0.504,22.6
1,Sierra,Magneuris,642423,146,CF,63,3.91,30.2,sierra,magneuris,...,-3.16,NaN,0.411,0.615,0.513,0.595,0.901,0.777,0.496,22.0
2,Quinn,Roman,596451,143,CF,56,3.88,30.2,quinn,roman,...,0.94,NaN,0.235,0.696,0.479,0.452,0.830,0.743,0.529,22.4
3,Engel,Adam,641553,145,CF,181,4.08,30.1,engel,adam,...,-2.48,-2.79,0.394,0.669,0.528,0.513,0.823,0.704,0.486,24.0
4,Hamilton,Billy,571740,113,CF,203,3.94,30.1,hamilton,billy,...,-1.44,-17.84,0.303,0.642,0.476,0.692,0.826,0.784,0.510,21.5


In [ ]:
plt.scatter(CFers.sprint_speed, CFers.Fld)

In [ ]:
plt.scatter(total2018.sprint_speed, total2018.UBR)


In [10]:
from sqlalchemy import create_engine

In [27]:
rds_connection_string = "root:147258@localhost:3306/baseballdb"
engine = create_engine(f'mysql+pymysql://{rds_connection_string}')

In [28]:
total2018.to_sql('BaseballStats', con=engine)

InternalError: (pymysql.err.InternalError) (1049, "Unknown database 'baseballdb'") (Background on this error at: http://sqlalche.me/e/2j85)